## import packages

In [1]:
import re
import os
import sys
import csv
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta

In [2]:
print("Hello")

Hello


In [3]:

from django.http import HttpResponse
from django.shortcuts import render
import json
import codecs
import socket
import webbrowser
import urllib.request
import datetime
import pytz

import csv
from django.http import StreamingHttpResponse
import os
import sys
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta
from time import sleep
import numpy as np
import math
import csv
import re
import os
import sys
import pandas as pd
import datetime
from datetime import timedelta

In [11]:
# calculate Gaussian Mixture Probability
def calculate_gaussian(len_time, w, m, d):
    #print("calculate_gaussian start!")
    prob = []

    listProbabilities = []
    for i in range(len(w)):
        for j in range(len_time):
            #print(type(m))
            #print(type(d))
            u = ((j - m[i]) / np.abs(d[i]))
            #print(u)
            y = (1 / (math.sqrt(2 * math.pi) * np.abs(d[i]))) * math.exp(-u * u / 2)
            #print(y)
            #print("prob: " , y)
            prob.append(y)

        for k in range(len_time):
            probability = prob[k] * w[i]
            listProbabilities.append(probability)

    return listProbabilities

#rescale (gaussian prob -> maximum and minimum values by season)
def rescale(values, new_min, new_max):
    rescale_output = []
    old_min, old_max = min(values), max(values)

    for v in values:
        #rescale
        new_v = (new_max - new_min) / (old_max - old_min) * (v - old_min) + new_min
        #rounds
        round_new_v = round(new_v, 2)
        rescale_output.append(round_new_v)

    return rescale_output

def temperatureDustDataGenerate():
    #Set maximum and minimum values by season
    springMin = 15
    springMax = 25

    summerMin = 25
    summerMax = 35

    fallMin = 15
    fallMax = 25

    winterMin = -10
    winterMax = 5


    #need to fix global variables
    temperature_weight = [1]
    temperature_mean = [840.0] # 14:00
    temperature_dev = [150.0]

    finedust_weight = [0.6,0.4]
    finedust_mean = [660.0, 1020.0] #11:00, 17:00
    finedust_dev = [150.0, 90.0]

        #print("TV calculateGaussian: ", calculateGaussian(TV_weight, TV_mean, TV_dev))
    temp_prob = calculate_gaussian(1440, temperature_weight, temperature_mean, temperature_dev)
    dust_prob = calculate_gaussian(1440, finedust_weight, finedust_mean, finedust_dev)

    #initialize rescale values
    temp_spring = rescale(temp_prob, springMin, springMax)
    temp_summer = rescale(temp_prob, summerMin, summerMax)
    temp_fall = rescale(temp_prob, fallMin, fallMax)
    temp_winter = rescale(temp_prob, winterMin, winterMax)

    dust_spring = rescale(dust_prob, springMin, springMax)
    dust_summer = rescale(dust_prob, summerMin, summerMax)
    dust_fall = rescale(dust_prob, fallMin, fallMax)
    dust_winter = rescale(dust_prob, winterMin, winterMax)


#consider threshold (Aircon, Aircleaner)
def threshold_generator(season, threshold_temp, threshold_dust):
    if season.lower() == "spring":
        Temperature = temperatureDustDataGenerate().temp_spring
        Fine_dust = temperatureDustDataGenerate().dust_spring
    if season.lower() == "summer":
        Temperature = temperatureDustDataGenerate().temp_summer
        Fine_dust = temperatureDustDataGenerate().dust_summer
    if season.lower() == "fall":
        Temperature = temperatureDustDataGenerate().temp_fall
        Fine_dust = temperatureDustDataGenerate().dust_falll
    if season.lower() == "winter":
        Temperature = temperatureDustDataGenerate().temp_winter
        Fine_dust = temperatureDustDataGenerate().dust_winter

    Temp_1and0 = []
    Dust_1and0 = []

    Temp_onoff = []
    Dust_onoff = []

    for i in range(len(Temperature)):
        if Temperature[i]>=threshold_temp:
            Temp_1and0.append(1)
            Temp_onoff.append('On')
        else:
            Temp_1and0.append(0)
            Temp_onoff.append('Off')

        if Fine_dust[i]>=threshold_dust:
            Dust_1and0.append(1)
            Dust_onoff.append('On')
        else:
            Dust_1and0.append(0)
            Dust_onoff.append('Off')

# Data Generator
def gaussian_generetor(user, start, end, days, season, week, weather, device, weight, mean, dev):
    print("gaussian_generetor start!")
    #col0 Time data generate. (24*60 = 1440 minute)
    s = datetime.datetime.strptime(start, '%H:%M')
    e = datetime.datetime.strptime(end, '%H:%M')

    time =[]
    while s < e:
        time.append(s)
        s += datetime.timedelta(minutes=1)

    time = [t.strftime("%H:%M") for t in time]

    #print(s, e)
    #print(time)

    #col1 User_id data generate. (1->kim, 2->park)

    #col2 Device data generate. (TV, Bulb, RobotCleaner, AirCleaner, AirConditioner)

    #col3 n-components data generate. (number of gaussian mixtures)
    n_com = str(len(weight))

    #col4 Gaussian Mixture Probability data generate. (extract to calculateGaussian Function)
    prob = []
    for i in range(len(time)):
        prob.append(calculate_gaussian(len(time), weight, mean, dev)[i])
    #print(prob)

    #col5 ON/OFF data generate.
    on_off = []
    for i in range(len(time)):
        if(i<420):
            on_off.append("Off")
        else:
            on_off.append("On")

    #col6 Season data generate. (Spring, Summer, fall, Winter)

    #col7 Week data generate. (Weekdays, Weekend)

    #col8 Weather data generate. (Sunny, Blur, Rainy)

    # write the csv file
    #root_dir = os.path.join(BASE_DIR, "polls/static/new_data/")
    root_dir = "./code_test"
    
    print("Generate " + device + " Data Start!")
    for i in range(days):
        write_loc = root_dir + "/" + user.lower() + "/" + season.lower() + "/" + week.lower() + "/" + device.lower() + "/"+ str(i) + ".csv"
        print(write_loc)
        f = open(write_loc, 'w', encoding='utf-8', newline='')
        wr = csv.writer(f, delimiter='\t')

        for i in range(len(time)):
            wr.writerow([time[i], user, device, n_com, prob[i], on_off[i], season, week, weather])

        f.close()
    
    print("Generate " + device + " Data End!")



def make_dir():
    root_dir = "./code_test"
    user_list = ['park', 'kim', 'choi']
    device_list = ['airconditioner', 'aircleaner', 'bulb', 'robotcleaner', 'tv']
    season_list = ['spring', 'summer', 'fall', 'winter']
    week_list = ['weekdays', 'weekend']
    weather_list = ['sunny', 'blur', 'rainy']
    choose_list = []

    for i in range(len(user_list)):
        for j in range(len(season_list)):
            for k in range(len(week_list)):
                for n in range(len(device_list)):
                    choose_list.append(
                        root_dir + "/" + user_list[i] + "/" + season_list[j] + "/"
                        + week_list[k] + "/" + device_list[n])

    for i in choose_list:
        if not os.path.exists(i):
            os.makedirs(i)

######## @jaeseung : need to fix
def generate_data(user, start, end, days, season, week, weather,
                  tv_weight, tv_mean, tv_dev,
                  bulb_weight, bulb_mean, bulb_dev, 
                  robot_weight, robot_mean, robot_dev,
                  aircon_thres, aircleaner_thres):
    
    device = ["Airconditioner", "Aircleaner", "Bulb", "Robotcleaner", "TV"]

    make_dir()
    print("Make Directory Structure Finished!")
    gaussian_generetor(user, start, end, days, season, week, weather, device[2], bulb_weight, bulb_mean, bulb_dev)
    print("Bulb Data Generated!")
    gaussian_generetor(user, start, end, days, season, week, weather, device[3], robot_weight, robot_mean, robot_dev)
    print("RobotCleaner Data Generated!")
    gaussian_generetor(user, start, end, days, season, week, weather, device[4], tv_weight, tv_mean, tv_dev)
    print("TV Data Generated!")
    #threshold_generator(season, threshold_temp, threshold_dust)
    


In [12]:
user = "Park"
start = "8:00"
end = "12:00"
days = 3
season = "Fall"
week = "Weekend"
weather = "Sunny"

tv_weight  = [0.6, 0.4]
tv_mean= [700.0, 1000.0]
tv_dev = [30.0, 50.0]

bulb_weight= [0.3, 0.7]
bulb_mean = [1000.0, 1200.0]
bulb_dev  = [40.0, 50.0]
robot_weight = [0.6, 0.4]
robot_mean = [500.0, 800.0]
robot_dev = [30.0, 40.0]

#수정해야할듯.. threshold 적용
AirCleaner_weight = [0.6, 0.4]
AirCleaner_mean = [700.0, 1100.0]
AirCleaner_dev = [30.0, 50.0]

#수정해야할듯.. threshold 적용
AirConditioner_weight = [0.4, 0.6]
AirConditioner_mean = [700.0, 1100.0]
AirConditioner_dev = [40.0, 50.0]
#print("TV calculateGaussian: ", calculateGaussian(TV_weight, TV_mean, TV_dev))

aircon_thres= 25
aircleaner_thres = 50

generate_data(user, start, end, days, season, week, weather,
              tv_weight, tv_mean ,tv_dev,
              bulb_weight ,bulb_mean ,bulb_dev,
              robot_weight ,robot_mean ,robot_dev,
              aircon_thres, aircleaner_thres)

Make Directory Structure Finished!
gaussian_generetor start!
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'l

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

...

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'li

KeyboardInterrupt: 

In [15]:

tv_weight  = "0.6, 0.4"

print(tv_weight.split())

['0.6,', '0.4']


In [ ]:

def temperatureDataGenerate():
    #Set temperature maximum and minimum values by season
    tempSpringMin = 15
    tempSpringMax = 25

    tempSummerMin = 25
    tempSummerMax = 35

    tempFallMin = 15
    tempFallMax = 25

    tempWinterMin = -10
    tempWinterMax = 5

    #initialize rescale values
    temp_spring = rescale(temp_prob, tempSpringMin, tempSpringMax)
    temp_summer = rescale(temp_prob, tempSummerMin, tempSummerMax)
    temp_fall = rescale(temp_prob, tempFallMin, tempFallMax)
    temp_winter = rescale(temp_prob, tempWinterMin, tempWinterMax)


def finedustDataGenerate():
    #Set fine dust maximum and minimum values by season

    dustSpringMin = 35
    dustSpringMax = 70

    dustSummerMin = 25
    dustSummerMax = 45

    dustFallMin = 25
    dustFallMax = 45

    dustWinterMin = 35
    dustWinterMax = 70

    #initialize rescale values
    dust_spring = rescale(temp_prob, dustSpringMin, dustSpringMax)
    dust_summer = rescale(temp_prob, dustSummerMin, dustSummerMax)
    dust_fall = rescale(temp_prob, dustSummerMax, dustFallMin)
    dust_winter = rescale(temp_prob, dustWinterMin, dustWinterMax)


#consider threshold (Aircon, Aircleaner)
def thresholdGenerator(s, e, user, season, week, weather, threshold_temp, threshold_dust):
    if season.lower() == "spring":
        Temperature = temperatureDataGenerate().temp_spring
        Fine_dust = finedustDataGenerate().dust_spring
    if season.lower() == "summer":
        Temperature = temperatureDataGenerate().temp_summer
        Fine_dust = finedustDataGenerate().dust_summer
    if season.lower() == "fall":
        Temperature = temperatureDataGenerate().temp_fall
        Fine_dust = finedustDataGenerate().dust_falll
    if season.lower() == "winter":
        Temperature = temperatureDataGenerate().temp_winter
        Fine_dust = finedustDataGenerate().dust_winter

    Temp_1and0 = []
    Dust_1and0 = []

    Temp_onoff = []
    Dust_onoff = []

    for i in range(len(Temperature)):
        if Temperature[i] >= threshold_temp:
            Temp_1and0.append(1)
            Temp_onoff.append('On')
        else:
            Temp_1and0.append(0)
            Temp_onoff.append('Off')

        if Fine_dust[i] >= threshold_dust:
            Dust_1and0.append(1)
            Dust_onoff.append('On')
        else:
            Dust_1and0.append(0)
            Dust_onoff.append('Off')
    
    
    time = []
    while s < e:
        time.append(s)
        s += datetime.timedelta(minutes=1)

    time = [t.strftime("%H:%M") for t in time]
    
    
    
    for i in range(days):

        write_loc = root_dir + user.lower() + "/" + season.lower() + "/" + week.lower() + "/" + "Airconditioner".lower() \
                    + "/" + str(i) + ".csv"
        print(write_loc)
        f = open(write_loc, 'w', encoding='utf-8', newline='')
        wr = csv.writer(f, delimiter='\t')

        for i in range(len(time)):
            wr.writerow([time[i], user, "Airconditioner", "1", Temperature[i], Temp_onoff[i], season, week, weather])

        f.close()

    for i in range(days):

        write_loc = root_dir + user.lower() + "/" + season.lower() + "/" + week.lower() + "/" + "Aircleaner".lower() \
                    + "/" + str(i) + ".csv"
        print(write_loc)
        f = open(write_loc, 'w', encoding='utf-8', newline='')
        wr = csv.writer(f, delimiter='\t')

        for i in range(len(time)):
            wr.writerow([time[i], user, "Aircleaner", "2", Fine_dust[i], Dust_onoff[i], season, week, weather])

        f.close()

    

In [ ]:
    #initialize rescale values
    temp_spring = rescale(temperatureDustInfo().temp_prob, tempSpringMin, tempSpringMax)
    temp_summer = rescale(temperatureDustInfo().temp_prob, tempSummerMin, tempSummerMax)
    temp_fall = rescale(temperatureDustInfo().temp_prob, tempFallMin, tempFallMax)
    temp_winter = rescale(temperatureDustInfo().temp_prob, tempWinterMin, tempWinterMax)
    #initialize rescale values
    dust_spring = rescale(temperatureDustInfo().dust_prob, dustSpringMin, dustSpringMax)
    dust_summer = rescale(temperatureDustInfo().dust_prob, dustSummerMin, dustSummerMax)
    dust_fall = rescale(temperatureDustInfo().dust_prob, dustSummerMax, dustFallMin)
    dust_winter = rescale(temperatureDustInfo().dust_prob, dustWinterMin, dustWinterMax)


def temperatureDustInfo():
    # need to fix global variables
    temperature_weight = [1]
    temperature_mean = [840.0]  # 14:00
    temperature_dev = [150.0]

    finedust_weight = [0.6, 0.4]
    finedust_mean = [660.0, 1020.0]  # 11:00, 17:00
    finedust_dev = [150.0, 90.0]

    # print("TV calculateGaussian: ", calculateGaussian(TV_weight, TV_mean, TV_dev))
    temp_prob = calculateGaussian(temperature_weight, temperature_mean, temperature_dev)
    dust_prob = calculateGaussian(finedust_weight, finedust_mean, finedust_dev)

In [29]:
start= "8:00"
x= "0:00"

s = datetime.datetime.strptime(start, '%H:%M')
x = datetime.datetime.strptime(x, '%H:%M')
new = s-x
print(int(new.seconds/60))

print(s.seconds)

480


AttributeError: 'datetime.datetime' object has no attribute 'seconds'

In [33]:
day_start = "0:00"

day_s = datetime.datetime.strptime(day_start, '%H:%M')
print(int(new.seconds / 60))

s = datetime.datetime.strptime("8:00", '%H:%M')
e = datetime.datetime.strptime("20:00", '%H:%M')

time = []
min_list = []
while s < e:
    time.append(s)
    min_list.append((s - day_s).seconds/60)
    s += datetime.timedelta(minutes=1)

time = [t.strftime("%H:%M") for t in time]

print(time)
print(min_list)

480
['08:00', '08:01', '08:02', '08:03', '08:04', '08:05', '08:06', '08:07', '08:08', '08:09', '08:10', '08:11', '08:12', '08:13', '08:14', '08:15', '08:16', '08:17', '08:18', '08:19', '08:20', '08:21', '08:22', '08:23', '08:24', '08:25', '08:26', '08:27', '08:28', '08:29', '08:30', '08:31', '08:32', '08:33', '08:34', '08:35', '08:36', '08:37', '08:38', '08:39', '08:40', '08:41', '08:42', '08:43', '08:44', '08:45', '08:46', '08:47', '08:48', '08:49', '08:50', '08:51', '08:52', '08:53', '08:54', '08:55', '08:56', '08:57', '08:58', '08:59', '09:00', '09:01', '09:02', '09:03', '09:04', '09:05', '09:06', '09:07', '09:08', '09:09', '09:10', '09:11', '09:12', '09:13', '09:14', '09:15', '09:16', '09:17', '09:18', '09:19', '09:20', '09:21', '09:22', '09:23', '09:24', '09:25', '09:26', '09:27', '09:28', '09:29', '09:30', '09:31', '09:32', '09:33', '09:34', '09:35', '09:36', '09:37', '09:38', '09:39', '09:40', '09:41', '09:42', '09:43', '09:44', '09:45', '09:46', '09:47', '09:48', '09:49', '09:5

In [39]:
new = np.arange(1440)
print(len(new))
print(len(min_list))
print(len(new[int(min_list[0]):int(min_list[-1]+1)]))

1440
720
720


In [40]:
print()